# [Attention is All You Need](https://arxiv.org/abs/1706.03762) Review

* 이전까지의 Transduction 모델에서는 recurrent NN 또는 convolutional NN 에서 인코더와 디코더를 attention mechanism을 사용해 연결한 구조를 사용
* Transformer 모델은 attention mechanism에만 기반한 모델
* 두 번의 기계번역 실험에서 해당 모델은 병렬화, 학습시간 감소와 함께 높은 성능을 보임

## Introduction
* RNN, LSTM, GRU는 sequence modeling과 transduction 문제들에 높은 성능을 보여주었다
* Recurrent 모델 
    * 입력, 출력의 symbolic position에 따라 계산 수행
    * 계산 위치 맞추기 위해 은닉 상태 h_t 를 생성 
    * **but** 메모리 제한으로 sequence 가 길어질수록 병렬화가 힘들어진다
* Attention Mechanism
    * 입출력 sequence의 거리와 상관없이 dependency modeling을 가능하게 함
    * **but** 아직까지는 대부분 recurrent network와 함께 사용되고 있다
* 해당 논문에서는 RNN 또는 CNN을 사용하지 않고 self-attention에만 의존하여 입력-출력 사이의 의존성을 학습하는 Transformer 모델을 제안한다

## Background
* Extended Neural GPU, ByteNet, ConvS2S
    * Sequential 연산 줄이기 위해 제안 -> hidden representation을 병렬계산
    * CNN 활용
    * position의 거리가 길 수록 연산량 증가
    * distant 한 position 사이의 의존성 학습 어려움
* Transformer -> O(n)의 시간복잡도 내로 계산 가능해짐
    * attention-weighted position의 평균을 계산
* Self-attention: sequence representation을 계산하기 위해 한 sequence 내의 다른 position 연결
* Transformer는 RNN 또는 convolution 없이 self-attention만으로 input과 ouput representation을 계산하는 첫 모델이다

## Model Architecture
* sequence transdction model 들은 대부분 같이 encoder-decoder 구조를 활용한다
* Encoder: 입력 symbol representation(x1, ... xn)을 continuous representations z = (z1, ... zn)으로 매핑한다
* Decoder: continuous representation z가 주어질 때 한 번에 한 원소씩 output sequence(y1, ... yn)를 생성한다
* 각 스텝에서 모델은 auto-regressive 하며 이전 단계에서 생성된 symbol을 additional input으로 받아들여 다음 단계의 symbol을 생성할 때 활용한다
* Transformer 모델은 encoder-decoder에서 stacked self-attention과 point-wise, fully connected layer를 사용한다
![alt-text](https://miro.medium.com/max/2400/1*Ir-j0HTT-IoWcZSyaE2F-A.png)

### Encoder and Decoder Stacks

#### Encoder
* N=6 개의 동일한 layer 들이 스택되어 있다
* 각 layer 2개의 sub layer 로 이루어져 있다 
    * multi-head self-attention mechanism
    * point-wise fully connected layer -> simple feed-forward network
* 각 sub-layer에 residual connection 연결(연결 용이하게 하기 위해 embedding layer 포함한 모든 sub-layer는 d = 512 차원의 출력값 가짐) 

#### Decoder
* N=6 개의 동일한 layer들로 구성
* 각 layer에 Encoder와 동일한 2 sub-layer 외 1개 추가
    * encoder stack의 output에 multi-head attention을 수행하는 sub-layer
* 인코더와 유사한 형태로 residual connection이 각 sub-layer의 normalization layer 뒤에 따라옴
* 출력 생성시 다음 출력에서 정보 얻는 것 방지 위해 masking 사용 
    -> i 번째 원소 생성새 1~(i-1) 원소만 참조 가능하도록 함

### Attention
![alt-text](https://miro.medium.com/max/948/1*GDp8g6XFCJ7u0aK60-q4kA@2x.png)
* Attention function은 query 와 key value set을 output으로 mapping 한다
* query, key, value, output 은 모두 벡터
* output -> value들의 weighted sum으로 계산 
* weight 는 query와 연관된 key의 compatibility function에 의해 계산된다

#### Scaled Dot-Product Attention

* attention 계산시 dot-product 사용, 결과에 scaling factor 추가
* 입력: d_k 차원의 query와 key, d_v 차원의 value로 구성
* query 와 key 의 dot product 계산 -> root(d_k) 로 나눈 후 softmax를 적용해 value의 가중치를 얻는다
* query 들을 동시에 계산하기 위해 matrix Q로 묶는다
* key, value 각각 matrix K, V로 표현
    Q, K, V의 attention 구하는 식:  
    $ Attention(Q, K, V) = \text{softmax} \Big( \frac{QK^T}{\sqrt{d_k}} \Big) V $

* 가장 널리 쓰이는 attention 함수:
    * Additive attention: single hidden layer의 feed-forward network 사용해 compatibility function 계산, d_k가 작을 때 성능이 좋음
    * Dot-product attention: d_k가 클 때 빠른 matrix multiplication algorithm에 의해 더 빠르고 공간 효율적

#### Multi-Head Attention

* query, key를 d_k 차원으로, value 를 d_v 차원으로 linear projection
* projected 된 버전의 query, key, value를 h번 학습(attention)
* 계산된 h 쌍의 d_v 차원을 concatenate 한 후 다시 projection 시킨다     
    식으로 나타낸 과정:     
    $ \text{MultiHead}(Q, K, V) = \text{Concat}(head_1, ..., head_h)W^O, where \ head_i=\text{Attention}(QW_i^Q, KW_i^K, VW_i^V)$ 
    
    논문에서는 $ h=8, d_k=d_v=d_{model}/h = 64 $ 사용
* 각 head의 차원이 줄어듦에 따라 total computational cost는 single-head attention과 비슷하다

#### Applications of Attention in out Model
* "encoder-decoder-attention" layer에서 query는 이전의 decoder layer에서 오고 memory key, value는 encoder의 output에서 온다. 이는 decoder의 모든 포지션에서 모든 input 포지션을 고려하게끔 한다
* encoder는 모든 key, value, query가 같은 곳에서 오는 self-attention layer를 가지므로 인코더의 각 포지션이 이전 layer의 모든 포지션을 고려할 수 있다
* decoder의 self-attention layer도 동일한 특징을 가지지만 auto-regressive 속성 보존을 위해 출력을 생성할 시 다음 출력을 고려하지 않기 위해 masking을 통해 이전 포지션은 참조할 수 없도록 한다.

### Position-wise Feed-Forward Networks
* 인코더와 디코더는 모든 position에 동일하게 적용되는 FC feed-forward 네트워크를 포함한다
* ReLU와 2개의 linear transformation으로 구성된다(kernel = 1인 CNN과 같다)
    $\text{FFN}(x) = \max(0, xW_1 + b_1)W_2 + b_2$     
    
    각 layer에 독립적인 parameter 적용, 논문에서는 $d_{model}=512, d_{ff} = 2048$ 사용

### Embedding and Softmax
* 2개의 embedding layer와 pre-softmax linear transformation에서 같은 weight 행렬 사용
* 임베딩 layer에서는 weight에 $\sqrt{d_{model}}$ 를 곱해준다

### Positional Encoding
* Transformer에는 RNN이나 CNN이 들어가 있지 않기에 sequence order 정보를 이용할 수 있도록 절대적/상대적인 포지션 정보를 넣어주어야 한다
--> 인코더 디코더 스택의 아래에 **positional encodings**를 input embedding에 추가하였다

## Why Self-Attention
* layer당 연산량이 적음
* 계산 병렬화 가능
    -> recurrent의 경우 순차적으로 진행되는 계산 때문에 병렬화 불가
* path 길이에 따른 long-range dependency
![alt-text](https://images1.programmersought.com/58/90/9052bbac4347f5467c6a0d4cb6d1bf9a.JPEG)
위 표에서 self-attention의 long-range dependency의 학습 속도가 가장 좋은 것을 확인할 수 있다

## Training
* optimizer: Adam (warmup step 동안에는 learning rate 증가, 이후 감소)
* Regularization:
    - Residual dropout
    - Label smoothing

## Results
![alt-text](https://miro.medium.com/max/2086/1*vezgbFHMEwnprctbglO8dA.png)
Machine Translation을 진행했을 때 이전 모델보다 높은 성능을 보였다

## Conclusion
* attention만을 사용한 encoder-decoder 모델
* 적은 계산량과 병렬화를 적용해 Recurrent NN, CNN 보다 빠른 학습 속도를 보임
* English -> German translation에서 앙상블 모델보다 뛰어난 성능을 보임

참고 자료   
https://greeksharifa.github.io/nlp(natural%20language%20processing)%20/%20rnns/2019/08/17/Attention-Is-All-You-Need/    
https://dladustn95.github.io/nlp/Transformer_paper_review/
